# Hypothesis Testing

Section: a. Older individuals prefer in-store shopping

**Part of:** [marketing_campaign_082825_working.ipynb](./marketing_campaign_082825_working.ipynb)

In [ ]:
# Setup and data loading
from utils import (
    ProjectConfig,
    load_intermediate_results,
    save_project_figure,
    reset_plot_settings,
)
import seaborn as sns
import matplotlib.pyplot as plt

config = ProjectConfig()
# Load data from previous notebook
df = load_intermediate_results('data_from_08_step_6.pkl', config)


## a. Older individuals prefer in-store shopping

In [ ]:
# a. Older individuals prefer in-store shopping

# Specify H₀ and H₁:
# H₀: The distributions of NumStorePurchases for older and younger customers are
#     identical (no difference in distribution).
# H₁: The distributions of NumStorePurchases for older customers is stochastically
#     greater (they tend to have higher values).

# Normality and Test Choice:
# •  Use the t-test if NumStorePurchases is approximately normally distributed.
# •  Use the Mann-Whitney U test if the data is non-normal.

# Interpreting Results:
# •  P-value ≤ α (0.05): Reject H₀, concluding there is evidence that older customers have
#    higher in-store purchases.
# •  P-value > α: Fail to reject H₀, indicating insufficient evidence that older customers
#    prefer in-store purchases.

In [ ]:
# Split data
older = df[df['Age'] >= df['Age'].median()]['NumStorePurchases']
younger = df[df['Age'] < df['Age'].median()]['NumStorePurchases']
df['Older'] = df['Age'] >= df['Age'].median()

In [ ]:
# Normality check
from scipy.stats import shapiro

# Purpose:
# •  Tests the null hypothesis (H₀) that the data from the groups is normally distributed.
# •  Alternative hypothesis (H₁): The data is not normally distributed.

for group, data in [(1, older), (0, younger)]:
    stat, p = shapiro(data)
    print(f"Shapiro-Wilk Test for Older Individuals = {group}: p-value = {p:.4f}")
    if p < 0.05:
        print("  p-value < 0.05: Reject H₀, data is not normally distributed.")
    else:
        print("  p-value >= 0.05: Fail to reject H₀, data may be normally distributed.")

In [ ]:
# Create a figure with two subplots side by side
reset_plot_settings()

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,6), sharey=True)

# Boxplot
sns.boxplot(x='Older', y='NumStorePurchases', data=df, ax=ax1, hue='Older', legend=False)
ax1.set_title("Box Plot of In-Store Purchases for Older Customers")
ax1.set_xlabel('Age')
ax1.set_xticks([0,1], ['Younger', 'Older'])
ax1.set_ylabel("Number of Store Purchases")
#ax1.set_ylim(-2,28)
ax1.grid(True, axis='y', linestyle='--', alpha=0.7)

# Violin Plot
sns.violinplot(x='Older', data=df, y='NumStorePurchases', legend=False, ax=ax2, hue='Older')
ax2.set_title("Violin Plot of In-Store Purchases for Older Customers")
ax2.set_xlabel('Age')
ax2.set_xticks([0,1], ['Younger', 'Older'])
ax2.grid(True, axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
save_project_figure(
    "Box_and_Violin_Plots_Older_Customers",
    "Box and Violin Plots of In-Store Purchases for Older Customers",
    config,
)
plt.show()

# Descriptive Statistics
stats = df.groupby('Older')['NumStorePurchases'].agg(['mean','median','std','count'])
print('\nDescriptive Statistics')
print(stats)

# Shapiro-Wilk test and plots show data is not normally distributed so a non-parametric
# test seems more appropriate

In [ ]:
# Perform Mann-Whitney U Test (One-Tailed)

# H₀: The distributions of NumStorePurchases for older and younger customers are
#     identical (no difference in distribution).
# H₁: The distributions of NumStorePurchases for older is stochastically
#     greater (they tend to have higher values).
from scipy.stats import mannwhitneyu

stat, p_value = mannwhitneyu(older, younger, alternative='greater')
print(f'Mann-Whitney U Test: statistic = {stat:.4f}, p-value = {p_value:.4f}')

# Interpretation
alpha = 0.05
if p_value <= alpha:
    print('Reject H₀: Older customer have higher in-store puchases')
else:
    print('Fail to reject H₀: No evidence that older customers have higher in-store purchases')

In [ ]:
# Why a t-test may be ok
# Large sample sizes (e.g., n > 30 or ideally n > 50 per group), due to the Central Limit Theorem (CLT).
# The CLT states that the sampling distribution of the mean approaches normality as sample size increases,
# even if the underlying data is non-normal.

print(f"Sample size for Older Customers:\n{df['Older'].value_counts()}")
# Sample Size is Large: Both groups (Older = 1 and 0) have n > 50
# the CLT helps ensure the t-test is robust.

In [ ]:
# Perform Levene's test

# Purpose:
# •  Tests the null hypothesis (H₀) that the variances of the groups are equal.
# •  Alternative hypothesis (H₁): The variances are not equal.

# Variations of test:
# •  Standard Levene’s Test: Uses the group mean for calculating deviations.
# •  Modified Levene’s Test (Brown-Forsythe): Uses the group median instead of the mean,
#    which is more robust to non-normal data.

# Implications of test:
# •  If Levene’s test indicates unequal variances (p-value ≤ 0.05), use Welch’s t-test to compare means.
# •  If variances are equal, use the standard t-test.

from scipy.stats import levene
alpha = 0.05

stat, p_levene = levene(older, younger, center='median')  # Use 'median' for Brown-Forsythe
print(f"Levene's Test: statistic = {stat:.4f}, p-value = {p_value:.4f}")

print(f"{'Unequal variances' if p_levene < alpha else 'Equal variances'} (p {'<' if p_levene < alpha else '>='} {alpha})")

In [ ]:
# Unequal variances use Welch’s t-test to compare means
# Run parametric Welch's t-test

from scipy.stats import ttest_ind
print("\nWelch's t-Test (unequal variances):")
stat, p_welch = ttest_ind(older, younger, alternative='greater', equal_var=False)
print(f"Welch's t-test: statistic = {stat:.4f}, p-value = {p_welch:.4f}")

if p_welch <= alpha:
    print('Reject H₀: Older customers have higher in-store puchases')
else:
    print('Fail to reject H₀: No evidence that older customers have higher in-store purchases')

In [ ]:
# Conclusion
# Both the parametric test (Welch's t-Test) and the non-parametric test (Mann-Whitney U Test)
# reject H₀, meaning there is evidence that older customers have more in-store purchases

In [ ]:

# Save results for next notebook
# save_intermediate_results(df_processed, 'processed_data.pkl', config)
# save_intermediate_results(analysis_results, 'analysis_results.pkl', config)
# print('✓ Results saved for next notebook')